# Amazon Nova Act SDK를 활용한 기본 브라우저 도구 사용

## 개요

이 튜토리얼에서는 Nova Act SDK를 사용하여 Amazon Bedrock Agentcore 브라우저 도구를 사용하는 방법을 알아봅니다. 브라우저 도구를 헤드리스로 사용하고 브라우저를 실시간으로 확인하는 방법을 예시로 제공합니다.

### 튜토리얼 세부 정보

| 정보 | 세부 정보 |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형 | 대화형 |
| 에이전트 유형 | 단일 |
| 에이전트 프레임워크 | Nova Act |
| LLM 모델 | Amazon Nova Act 모델 |
| 튜토리얼 구성 요소 | NovaAct를 사용하여 헤드리스 방식으로 브라우저 도구와 상호 작용 |
| 튜토리얼 수직 | 수직 |
| 예제 복잡성 | 쉬움 |
| 사용 SDK | Amazon BedrockAgentCore Python SDK, Nova Act |

### 튜토리얼 아키텍처

이 튜토리얼에서는 Nova Act를 브라우저 도구와 함께 사용하는 방법을 설명합니다.

이 예시에서는 Nova Act 에이전트에 자연어 명령을 전송하여 Bedrock Agentcore 브라우저에서 헤드리스 방식으로 작업을 수행합니다.

### 튜토리얼 주요 기능

* 헤드리스 방식으로 브라우저 도구 사용
* 브라우저 도구와 함께 Nova Act 사용

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다.
* Python 3.10 이상
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Nova Act SDK 및 API 키

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Bedrock Agentcore Browser 도구와 NovaAct 함께 사용하기
로컬에서 실행할 수 있는 Python 스크립트를 생성합니다. 스크립트에서 NovaAct는 브라우저 세션의 CDP 엔드포인트를 사용하여 해당 엔드포인트에 연결하고 Playwright 액추에이션을 수행합니다.

In [ ]:
%%writefile basic_browser_with_nova_act.py
"""Browser automation script using Amazon Bedrock AgentCore and Nova Act.

This script demonstrates AI-powered web automation by:
- Initializing a browser session through Amazon Bedrock AgentCore
- Connecting to Nova Act for natural language web interactions
- Performing automated searches and data extraction using browser
"""

from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
from rich.console import Console
import argparse
import json

console = Console()

from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("using region", region)

def browser_with_nova_act(prompt, starting_page, nova_act_key, region="us-west-2"):
    result = None
    with browser_session(region) as client:
        ws_url, headers = client.generate_ws_headers()
        try:
            with NovaAct(
                cdp_endpoint_url=ws_url,
                cdp_headers=headers,
                preview={"playwright_actuation": True},
                nova_act_api_key=nova_act_key,
                starting_page=starting_page,
            ) as nova_act:
                result = nova_act.act(prompt)
        except Exception as e:
            console.print(f"NovaAct error: {e}")
        finally:
            return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", required=True, help="Browser Search instruction")
    parser.add_argument("--starting-page", required=True, help="Starting URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API key")
    parser.add_argument("--region", default="us-west-2", help="AWS region")
    args = parser.parse_args()

    result = browser_with_nova_act(
        args.prompt, args.starting_page, args.nova_act_key, args.region
    )
    console.print(f"\n[cyan] Response[/cyan] {result.response}")
    console.print(f"\n[bold green]Nova Act Result:[/bold green] {result}")

#### Running the script
Paste your Nova Act API key below before running the script. 

In [ ]:
NOVA_ACT_KEY= '' ### Paste your Nova Act Key here

In [ ]:
!python basic_browser_with_nova_act.py --prompt "Search for macbooks and extract the details of the first one" --starting-page "https://www.amazon.com/" --nova-act-key {NOVA_ACT_KEY}

In [ ]:
!python basic_browser_with_nova_act.py --prompt "Extract and return Amazon revenue for the last 4 years" --starting-page "https://stockanalysis.com/stocks/amzn/financials/" --nova-act-key {NOVA_ACT_KEY}

## 무슨 일이 백그라운드에서 일어났을까요?

* `browser_session`을 사용하여 브라우저 클라이언트를 인스턴스화하면 브라우저 클라이언트가 생성되고 세션이 시작됩니다.
* 그런 다음 `cdp_endpoint_url`과 `cdp_headers`를 사용하여 Nova Act가 해당 브라우저 세션을 가리키도록 구성했습니다.
* Nova ACT SDK는 이제 자연어 명령을 사용하여 브라우저에서 Playwright 작동을 생성했습니다.

# Congratulations!